In [14]:
import pandas as pd
import numpy as np
import econtools.metrics as mt
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from itertools import *

In [17]:
df = pd.read_stata("data/output_data/All_Data_Base_EL.dta")

In [60]:
def treat_dummy(df):
    
    df = pd.get_dummies(df,columns=["treat"])
    df["treat"] = df["treat_Treatment"]
    df = df.drop(columns=["treat_Control"])
    df = df.drop(columns=["treat_Treatment"])
    
    return df

def table3_var(df):
    def control(ctrl):
        dummies = []
        nonlocal df
        ctrl_val  = df[ctrl].value_counts().keys().sort_values()
        for i in ctrl_val:
            dummies.append(ctrl+"_"+str(i))
        return dummies
    
    ctrl_col = ["std","perf_gap_used","C1_strat"]
    ctrl_list = list(map(control,ctrl_col))
    ctrl_list = list(chain.from_iterable(ctrl_list))
    df = pd.get_dummies(df,columns = ctrl_col)
    
    return df

def table3_reg(df, y):
    X = ["treat", "treat_ave", "ave", "ave_educ_ave", "educ_ave"]
    
    d = {}
    for i in y:
        result = mt.reg(df, i, X , fe_name = "school_code", cluster="hhid").summary
        d["table_{0}".format(i)] = result
    
    return d

In [50]:
#table3
df = pd.read_stata("data/output_data/All_Data_Base_EL.dta")

df = treat_dummy(df)

df["treat_ave_eudc_ave"] = df["treat"]*df["ave"]*df["educ_ave"]
df["ave_educ_ave"] = df["ave"]*df["educ_ave"]
df["treat_educ_ave"] = df["treat"]*df["educ_ave"]

df = table3_var(df)

d = table3_reg(df, ["retention", "EL_ln_tot_exp_notext", "attendance_rate"] )
d["table_retention"]

C:\Users\thxkn\anaconda3\lib\site-packages\pandas\io\stata.py:1409: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding of latin-1 is being used.  This can happen when a file
has been incorrectly encoded by Stata or some other software. You should verify
the string values returned are correct.
  warnings.warn(msg, UnicodeWarning)


In [ ]:
import re

_surrogates = re.compile(r"[\uDC80-\uDCFF]")

def detect_decoding_errors_line(l, _s=_surrogates.finditer):
    """Return decoding errors in a line of text

    Works with text lines decoded with the surrogateescape
    error handler.

    Returns a list of (pos, byte) tuples

    """
    # DC80 - DCFF encode bad bytes 80-FF
    return [(m.start(), bytes([ord(m.group()) - 0xDC00]))
            for m in _s(l)]

with open("data/output_data/All_Data_Base_EL.dta", encoding="utf8", errors="surrogateescape") as f:
    for i, line in enumerate(f, 1):
        errors = detect_decoding_errors_line(line)
        if errors:
            print(f"Found errors on line {i}:")
            for (col, b) in errors:
                print(f" {col + 1:2d}: {b[0]:02x}")

In [9]:
controls = ["educ_ave_reg", "educ_ave_miss", "female_reg", "female_miss", "primary_resp_fem"]
controls = controls + dummies
X = ["treat", "treat_ave", "ave", "ave_educ_ave", "educ_ave"]
y =["retention", "EL_ln_tot_exp_notext", "attendance_rate"]

retention_control = []
EL_ln_tot_exp_notext_control=["ln_tot_exp_notext_reg", "ln_tot_exp_notext_miss"] 
attendance_rate_control = ["attendance_sv"]

for endo in y:
    #controls = controls + (str(endo) + "_control")
    X = X + controls
    result = mt.reg(df, endo , X , fe_name = "school_code", cluster="hhid")
    df_result = result.summary
    df_result = df_result.drop(index=controls)

TypeError: can only concatenate list (not "str") to list

In [46]:
#얘로 써야겠다. X + dummies 만 해도 singular matrix 뜸
X = ["treat", "treat_ave", "ave", "ave_educ_ave", "educ_ave"]
result = mt.reg(df,"retention", X , fe_name = "school_code", cluster="hhid").summary
result
#df_result = df_result.drop(index=controls)

,coeff,se,t,p>t,CI_low,CI_high
treat,-5.253049,2.095217,-2.507162,0.012349,-9.365236,-1.140861
treat_ave,0.101240,0.036775,2.752944,0.006027,0.029063,0.173417
ave,0.017356,0.035749,0.485495,0.627445,-0.052807,0.087519
ave_educ_ave,-0.006574,0.004582,-1.434815,0.151694,-0.015566,0.002418
educ_ave,0.533659,0.278296,1.917592,0.055485,-0.012541,1.079858


In [37]:
X = ["treat", "treat_ave", "ave", "ave_educ_ave", "educ_ave"]
result = mt.reg(df,"attendance_rate", X , fe_name = "school_code", cluster="hhid").summary
result
#df_result = df_result.drop(index=controls)

,coeff,se,t,p>t,CI_low,CI_high
treat,-0.071183,2.649562,-0.026866,0.978571,-5.269818,5.127451
treat_ave,0.001315,0.049151,0.026750,0.978664,-0.095124,0.097753
ave,0.162971,0.051467,3.166519,0.001584,0.061989,0.263953
ave_educ_ave,-0.012094,0.007440,-1.625438,0.104349,-0.026692,0.002505
educ_ave,0.860158,0.434262,1.980735,0.047864,0.008104,1.712212


In [47]:
X = ["treat", "treat_ave", "ave", "ave_educ_ave", "educ_ave"]
result = mt.reg(df,"EL_ln_tot_exp_notext", X , fe_name = "school_code", cluster="hhid").summary
result
#df_result = df_result.drop(index=controls)

,coeff,se,t,p>t,CI_low,CI_high
treat,0.105852,0.116125,0.911533,0.362265,-0.122064,0.333768
treat_ave,-0.002193,0.002226,-0.985245,0.324776,-0.006562,0.002176
ave,0.001139,0.002370,0.480601,0.630920,-0.003512,0.005790
ave_educ_ave,0.000749,0.000353,2.122238,0.034098,0.000056,0.001442
educ_ave,0.005712,0.018465,0.309336,0.757139,-0.030529,0.041953


In [38]:
def calc_vif(X):

    # Calculating VIF
    vif = pd.DataFrame()
    vif["variables"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

    return(vif)

#check multicollinearity
df_colinear = df.drop(df.loc[df["ave_educ_ave"].isnull() == True].index)
#exog= X + controls
multi = df_colinear.loc[:,reg_endo]
vif = calc_vif(multi)



C:\Users\thxkn\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1687: RuntimeWarning: invalid value encountered in float_scalars
  return 1 - self.ssr/self.uncentered_tss
